In [2]:
import pyodbc
class Hotel():
    def Menu(self):
        self.conn = pyodbc.connect(
            'Driver={SQL Server};'
            'Server=LAPTOP-N8OKBLLT\\SQLEXPRESS;'
            'Database=Restorent;'
            'Trusted_Connection=yes;'
        )
        self.cursor = self.conn.cursor()

    def ShowMenu(self):
        print("------------- Menu -------------")
        self.cursor.execute("SELECT M_id,item_name,price from Menu1")
        for val in self.cursor:
            print(f"{val.M_id}.{val.item_name} - {val.price}")
    
    def TakeOrder(self):
        self.cursor.execute("SELECT ISNULL(MAX(order_id),0) + 1 FROM Orders")
        self.order_id = self.cursor.fetchone()[0]
        print("Your Order ID is:", self.order_id)
    
        while True:
            Item = input("Enter Item(or done):")
            if Item.lower() == 'done':
                break
            q = int(input("Enter quantity: "))
            self.cursor.execute(
            "select M_id,price from Menu1 where lower(item_name) = lower(?)",
                (Item,)
            )
            rst = self.cursor.fetchone()
            if rst:
                M_id = rst[0]
                price = rst[1]
                total = price * q
    
                self.cursor.execute(
                    "INSERT INTO Orders (order_id, M_id, quantity, total) VALUES (?, ?, ?, ?)",
                    (self.order_id, M_id, q, total)
                )
                self.conn.commit()
                print("Item Added Successfully ")
            else:
                print("Item not available ")
    
    def generatebill(self):
        print("-" * 70)
        print(" " * 20 + "Welcome to Nisarg Hotel")
        print("-" * 70)
        print(f"{'Sr':<10}{'Menu':<20}{'Quantity':<10}{'Price':<15}{'Total':<10}")
        print("-" * 70)

        self.cursor.execute("""
            SELECT m.item_name, o.quantity, m.price, o.total
            FROM Orders o
            JOIN Menu1 m 
            ON o.M_id = m.M_id
            WHERE o.order_id = ?
        """, self.order_id)

        final_Amount = 0
        sr = 1

        records = self.cursor.fetchall()

        for row in records:    
            print(f"{sr:<10}{row.item_name:<20}{row.quantity:<10}{row.price:<15.2f}{row.total:<10.2f}")
            final_Amount  += row.total
            sr += 1

        print("-" * 70)
        print(f"{' final Amount: ':>60}{ final_Amount:.2f}")
        print("-" * 70)

R = Hotel()
R.Menu()
R.ShowMenu()
R.TakeOrder()
R.generatebill()

------------- Menu -------------
1.Burger - 90.0
2.Pizza - 100.0
3.Cold Drink - 40.0
4.poolav - 70.0
5.poli bhagi - 80.0
6.tea - 10.0
Your Order ID is: 17


Enter Item(or done): burger
Enter quantity:  2


Item Added Successfully 


Enter Item(or done): pizza
Enter quantity:  2


Item Added Successfully 


Enter Item(or done): done


----------------------------------------------------------------------
                    Welcome to Nisarg Hotel
----------------------------------------------------------------------
Sr        Menu                Quantity  Price          Total     
----------------------------------------------------------------------
1         Burger              2         90.00          180.00    
2         Pizza               2         100.00         200.00    
----------------------------------------------------------------------
                                              final Amount: 380.00
----------------------------------------------------------------------
